In [ ]:
!wget https://raw.githubusercontent.com/marcin119a/PUMP2/main/long_dna.txt

--2023-07-24 19:13:32--  https://raw.githubusercontent.com/marcin119a/PUMP2/main/long_dna.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 797472 (779K) [text/plain]
Saving to: ‘long_dna.txt’

long_dna.txt        100%[===================>] 778.78K  --.-KB/s    in 0.02s   

2023-07-24 19:13:32 (47.3 MB/s) - ‘long_dna.txt’ saved [797472/797472]



In [ ]:
def count_lines_in_file(file_name):
    # Inicjalizacja licznika linii
    line_count = 0

    # Otwarcie pliku w trybie do odczytu
    with open(file_name, 'r') as file:
        for line in file:
            line_count += 1

    return line_count

file_name = 'long_dna.txt'
lines_count = count_lines_in_file(file_name)
print(f"Liczba linii w pliku '{file_name}': {lines_count}")


Liczba linii w pliku 'long_dna.txt': 29536


In [ ]:
def parse_data_file(data_path, num_samples):
    # Inicjalizacja list dla danych wejściowych i wyjściowych oraz zbiorów znaków
    input_texts = []
    target_texts = []
    input_characters = set()
    target_characters = set()

    # Wczytanie danych z pliku txt
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')

    # Przetwarzanie danych i zbiorów znaków
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text = line.split(' ')
        # Dodanie znaków startu i końca dla target_text
        target_text = '\t' + target_text + '\n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        # Aktualizacja zbiorów znaków
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)

    return input_texts, target_texts, input_characters, target_characters

num_samples = 20000
data_path = 'long_dna.txt'
input_texts, target_texts, input_characters, target_characters = parse_data_file(data_path, num_samples)

# Wyświetlenie informacji o danych
print("Liczba danych wejściowych:", len(input_texts))
print("Liczba danych wyjściowych:", len(target_texts))
print("Zbiór znaków wejściowych:", input_characters)
print("Zbiór znaków wyjściowych:", target_characters)


Liczba danych wejściowych: 20000
Liczba danych wyjściowych: 20000
Zbiór znaków wejściowych: {'N', 'T', 'A', 'G', 'C'}
Zbiór znaków wyjściowych: {'\t', 'N', 'T', '\n', 'A', 'G', 'C'}


In [ ]:
input_characters

{'A', 'C', 'G', 'N', 'T'}

In [ ]:
target_characters

{'\t', '\n', 'A', 'C', 'G', 'N', 'T'}

In [ ]:
from __future__ import print_function
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np



latent_dim = 256  # Latent dimensionality of the encoding space.

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
import numpy as np

def create_token_index(tokens):
    token_index = dict([(char, i) for i, char in enumerate(tokens)])
    return token_index

def transform_data(input_texts, target_texts, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length):
    num_samples = len(input_texts)
    num_encoder_tokens = len(input_token_index)
    num_decoder_tokens = len(target_token_index)

    encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype='float32')
    decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype='float32')

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.
        for t, char in enumerate(target_text):
            decoder_input_data[i, t, target_token_index[char]] = 1.
            if t > 0:
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.

    return encoder_input_data, decoder_input_data, decoder_target_data

# Przykład użycia funkcji
input_token_index = create_token_index(input_characters)
target_token_index = create_token_index(target_characters)
encoder_input_data, decoder_input_data, decoder_target_data = transform_data(input_texts, target_texts, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)

# Wyświetlenie informacji o danych
print("Number of unique input tokens:", len(input_token_index))
print("Number of unique output tokens:", len(target_token_index))
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 20000
Number of unique input tokens: 5
Number of unique output tokens: 7
Max sequence length for inputs: 12
Max sequence length for outputs: 14
Number of unique input tokens: 5
Number of unique output tokens: 7
Max sequence length for inputs: 12
Max sequence length for outputs: 14


In [ ]:
np.savez_compressed('decoder_target_data.npz', decoder_target_data=decoder_target_data)

In [ ]:
np.savez_compressed('decoder_input_data.npz', decoder_input_data=decoder_input_data)

In [ ]:
np.savez_compressed('encoder_input_data.npz', encoder_input_data=encoder_input_data)

In [ ]:
batch_size = 128  # Batch size for training.
epochs = 100  # Number of epochs to train for.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer=Adam(learning_rate=0.005), loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')


Epoch 1/100
125/125 [==============================] - 13s 14ms/step - loss: 1.1962 - accuracy: 0.3943 - val_loss: 0.9889 - val_accuracy: 0.4925
Epoch 2/100
125/125 [==============================] - 1s 8ms/step - loss: 0.7753 - accuracy: 0.6102 - val_loss: 0.4895 - val_accuracy: 0.7461
Epoch 3/100
125/125 [==============================] - 1s 10ms/step - loss: 0.3911 - accuracy: 0.7909 - val_loss: 0.2704 - val_accuracy: 0.8391
Epoch 4/100
125/125 [==============================] - 1s 12ms/step - loss: 0.2199 - accuracy: 0.8586 - val_loss: 0.1719 - val_accuracy: 0.8727
Epoch 5/100
125/125 [==============================] - 1s 7ms/step - loss: 0.1443 - accuracy: 0.8778 - val_loss: 0.1291 - val_accuracy: 0.8798
Epoch 6/100
125/125 [==============================] - 1s 7ms/step - loss: 0.1408 - accuracy: 0.8751 - val_loss: 0.1308 - val_accuracy: 0.8778
Epoch 7/100
125/125 [==============================] - 1s 8ms/step - loss: 0.1200 - accuracy: 0.8798 - val_loss: 0.1165 - val_accuracy: 0.

In [ ]:

def matching_position_k_mers(result1, result2, space = True):

  # Initialise counter of compatible items
  matching_positions = 0
  if space == True:
    result2 = ' ' + result2
  # Compare the letters in each position
  for i in range(len(result1)):
      if result1[i] == result2[i]:
          matching_positions += 1

  return {matching_positions}

matching_position_k_mers('GTTTTGCAGCCG', 'TTTTGCAGCCGA')


{11}

In [ ]:

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('On how many positions they agree on', matching_position_k_mers(input_texts[seq_index], decoded_sentence))

1/1 [==============================] - 1s 570ms/step
-
Input sentence: GTTTTGCAGCCG
Decoded sentence: TTTTGCAGCCGA

On how many positions they agree on {11}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: TTTTGCAGCCGA
Decoded sentence: TTTGCAGCCGAC

On how many positions they agree on {11}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: TTTGCAGCCGAT
Decoded sentence: TTGCAGCCGATC

On how many positions they agree on {11}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: TTGCAGCCGATC
Decoded sentence: TGCAGCCGATCA

On how many positions they agree on {11}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: TGCAGCCGATCA
Decoded sentence: GCAGCCGATCAC

On how many positions they agree on {11}
1/1 [==============================] - 0s 17ms/step
-
Input sentence: GCAGCCGATCAT
Decoded sentence: CAGCCGATCATC

On how many positions they agree on {11}
1/1 [==============================] - 0s 18ms/step
-
Input sen

In [ ]:
!wget https://raw.githubusercontent.com/marcin119a/PUMP2/main/long_dna_errors.txt

--2023-07-24 19:16:02--  https://raw.githubusercontent.com/marcin119a/PUMP2/main/long_dna_errors.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 766688 (749K) [text/plain]
Saving to: ‘long_dna_errors.txt’

long_dna_errors.txt 100%[===================>] 748.72K  --.-KB/s    in 0.02s   

2023-07-24 19:16:02 (43.7 MB/s) - ‘long_dna_errors.txt’ saved [766688/766688]



### Corrected ERRORS by trained encoder-decoder network

In [ ]:
file_name = 'long_dna_errors.txt'
lines_count = count_lines_in_file(file_name)
print(f"Liczba linii w pliku '{file_name}': {lines_count}")

Liczba linii w pliku 'long_dna_errors.txt': 29488


In [ ]:
num_samples = 20000
data_path = 'long_dna_errors.txt'
input_texts, target_texts, input_characters, target_characters = parse_data_file(data_path, num_samples)

# Wyświetlenie informacji o danych
print("Liczba danych wejściowych:", len(input_texts))
print("Liczba danych wyjściowych:", len(target_texts))
print("Zbiór znaków wejściowych:", input_characters)
print("Zbiór znaków wyjściowych:", target_characters)


Liczba danych wejściowych: 20000
Liczba danych wyjściowych: 20000
Zbiór znaków wejściowych: {'T', 'A', 'N', 'G', 'C'}
Zbiór znaków wyjściowych: {'\t', 'T', 'A', '\n', 'N', 'G', 'C'}


In [ ]:
from __future__ import print_function
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

input_token_index = create_token_index(input_characters)
target_token_index = create_token_index(target_characters)
encoder_input_data, decoder_input_data, decoder_target_data = transform_data(input_texts, target_texts, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)

# Wyświetlenie informacji o danych
print("Number of unique input tokens:", len(input_token_index))
print("Number of unique output tokens:", len(target_token_index))
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)




Number of unique input tokens: 5
Number of unique output tokens: 7
Max sequence length for inputs: 12
Max sequence length for outputs: 14


In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=300,
          validation_split=0.2)

Epoch 1/300
125/125 [==============================] - 1s 8ms/step - loss: 1.6245 - accuracy: 0.4134 - val_loss: 0.9500 - val_accuracy: 0.5447
Epoch 2/300
125/125 [==============================] - 1s 8ms/step - loss: 0.8438 - accuracy: 0.5966 - val_loss: 0.7679 - val_accuracy: 0.6293
Epoch 3/300
125/125 [==============================] - 1s 8ms/step - loss: 0.6972 - accuracy: 0.6721 - val_loss: 0.6582 - val_accuracy: 0.6949
Epoch 4/300
125/125 [==============================] - 1s 7ms/step - loss: 0.6045 - accuracy: 0.7191 - val_loss: 0.5842 - val_accuracy: 0.7259
Epoch 5/300
125/125 [==============================] - 1s 10ms/step - loss: 0.5396 - accuracy: 0.7487 - val_loss: 0.5222 - val_accuracy: 0.7615
Epoch 6/300
125/125 [==============================] - 1s 12ms/step - loss: 0.4996 - accuracy: 0.7651 - val_loss: 0.4927 - val_accuracy: 0.7720
Epoch 7/300
125/125 [==============================] - 1s 8ms/step - loss: 0.4684 - accuracy: 0.7763 - val_loss: 0.4668 - val_accuracy: 0.78

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s_erros.h5')

Epoch 1/100
125/125 [==============================] - 1s 8ms/step - loss: 0.0702 - accuracy: 0.9108 - val_loss: 0.3994 - val_accuracy: 0.8445
Epoch 2/100
125/125 [==============================] - 1s 7ms/step - loss: 0.0675 - accuracy: 0.9118 - val_loss: 0.3967 - val_accuracy: 0.8451
Epoch 3/100
125/125 [==============================] - 1s 8ms/step - loss: 0.0659 - accuracy: 0.9128 - val_loss: 0.3957 - val_accuracy: 0.8459
Epoch 4/100
125/125 [==============================] - 1s 8ms/step - loss: 0.0658 - accuracy: 0.9129 - val_loss: 0.3936 - val_accuracy: 0.8459
Epoch 5/100
125/125 [==============================] - 1s 7ms/step - loss: 0.0663 - accuracy: 0.9123 - val_loss: 0.3943 - val_accuracy: 0.8461
Epoch 6/100
125/125 [==============================] - 1s 8ms/step - loss: 0.0655 - accuracy: 0.9129 - val_loss: 0.3903 - val_accuracy: 0.8467
Epoch 7/100
125/125 [==============================] - 1s 8ms/step - loss: 0.0708 - accuracy: 0.9101 - val_loss: 0.3932 - val_accuracy: 0.8469

In [ ]:

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())



for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Ground truth:', target_texts[seq_index])
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('On how many positions they agree on', matching_position_k_mers(target_texts[seq_index], decoded_sentence))

1/1 [==============================] - 0s 342ms/step
-
Ground truth: 	NTTTTGCAGCCG

Input sentence: NTTTTGCAGCCG
Decoded sentence: NTTTTGCAGCCG

On how many positions they agree on {13}
1/1 [==============================] - 0s 33ms/step
-
Ground truth: 	TTTTGCAGCCGA

Input sentence: TTNTGCAGCCGA
Decoded sentence: TTTTGCAGCCGC

On how many positions they agree on {12}
1/1 [==============================] - 0s 30ms/step
-
Ground truth: 	TTTGCAGCCGAT

Input sentence: TTTGCAGCCGAN
Decoded sentence: TTTGCAGCCGAT

On how many positions they agree on {13}
1/1 [==============================] - 0s 19ms/step
-
Ground truth: 	TTGCAGCCGATC

Input sentence: TTNCAGCCGATC
Decoded sentence: TTGCAGCCGATC

On how many positions they agree on {13}
1/1 [==============================] - 0s 20ms/step
-
Ground truth: 	TGCAGCCGATCA

Input sentence: TGCANCCGATCA
Decoded sentence: TGCAGCCGATCC

On how many positions they agree on {12}
1/1 [==============================] - 0s 18ms/step
-
Ground truth: 	GCAGC

In [ ]:
def filter_lines_with_character(input_file, output_file, character='N'):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            # Podziel linię na kolumny (oddzielone spacją)
            column1, column2 = line.strip().split(' ')
            # Sprawdź, czy w którejś kolumnie występuje znak 'N'
            if character in column1 or character in column2:
                # Jeśli tak, zapisz linię w pliku wynikowym
                f_out.write(line)

# Przykład użycia funkcji
input_file = 'long_dna.txt'
output_file = 'errors.txt'
filter_lines_with_character(input_file, output_file, character='N')


In [ ]:
num_samples = 10000
data_path = 'errors.txt'
input_texts, target_texts, input_characters, target_characters = parse_data_file(data_path, num_samples)

# Wyświetlenie informacji o danych
print("Liczba danych wejściowych:", len(input_texts))
print("Liczba danych wyjściowych:", len(target_texts))
print("Zbiór znaków wejściowych:", input_characters)
print("Zbiór znaków wyjściowych:", target_characters)


Liczba danych wejściowych: 53
Liczba danych wyjściowych: 53
Zbiór znaków wejściowych: {'N', 'T', 'A', 'G', 'C'}
Zbiór znaków wyjściowych: {'\t', 'N', 'T', '\n', 'A', 'G', 'C'}


In [ ]:
from __future__ import print_function
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

input_token_index = create_token_index(input_characters)
target_token_index = create_token_index(target_characters)
encoder_input_data, decoder_input_data, decoder_target_data = transform_data(input_texts, target_texts, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)

# Wyświetlenie informacji o danych
print("Number of unique input tokens:", len(input_token_index))
print("Number of unique output tokens:", len(target_token_index))
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)




Number of unique input tokens: 5
Number of unique output tokens: 7
Max sequence length for inputs: 12
Max sequence length for outputs: 14


In [ ]:
for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('On how many positions they agree on', matching_position_k_mers(input_texts[seq_index], decoded_sentence, space = False))

1/1 [==============================] - 0s 21ms/step
-
Input sentence: ACGGAACGTTCT
Decoded sentence: CCGGACGCAACA

On how many positions they agree on {5}
1/1 [==============================] - 0s 24ms/step
-
Input sentence: CGGAACGTTCTN
Decoded sentence: CGGTGCGAACAT

On how many positions they agree on {6}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: GGAACGTTCTNA
Decoded sentence: GGCTCGAACATG

On how many positions they agree on {5}
1/1 [==============================] - 0s 20ms/step
-
Input sentence: GAACGTTCTNAA
Decoded sentence: GCTCGAACATCTA

On how many positions they agree on {4}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: AACGTTCTNAAA
Decoded sentence: GACGAACATGTG

On how many positions they agree on {4}
1/1 [==============================] - 0s 19ms/step
-
Input sentence: ACGTTCTNAAAA
Decoded sentence: TCGAACATTGAT

On how many positions they agree on {4}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: